In [1]:
# Import Python package
import BinhoPulsar
from BinhoPulsar.Pulsar import Pulsar
from BinhoPulsar.definitions import *

## Getting started

#### 1. List all the Pulsar devices connected to the PC host

The ``BinhoPulsar.getConnectedPulsarDevicesList()`` gets a list of the Pulsar devices plugged into the host PC machine.

In [2]:
BinhoPulsar.getConnectedPulsarDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FD#7&384f89b9&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fd',
  'serial_number': '6CAA2A4ED8AB8A598C7E7777A8771F9',
  'release_number': 256,
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Pulsar',
  'usage_page': 65280,
  'usage': 1,
  'interface_number': 0}]

#### 2. Create an instance of the Pulsar class

To utilize a Pulsar USB host adapter device, we need to create an instance of the Pulsar class.

In [3]:
# Create a device instance. One instance of the Pulsar class represents a Pulsar USB host adapter device.
pulsar = Pulsar()

#### 3. Open connection to the Pulsar device

The public method ``Pulsar.open()`` establishes the connection with a Pulsar device. Below is the complete signature:

```python
open(vid, pid, serial, path)
```

- ``vid: int``: The Pulsar USB VID, which is set by default.
- ``pid: int``: The Pulsar USB PID, which is set by default.
- ``serial: str``: The Pulsar serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``BinhoPulsar.getConnectedPulsarDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Pulsar device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Pulsar device simultaneously.

In [4]:
# Use the method by default to connect to only one Pulsar device.
pulsar.open()

# Otherwise, use the path attribute to identify each Pulsar device. Uncomment the line below and comment the line #2.
# pulsar.open(path='\\\\?\\HID#VID_1FC9&PID_82FD#7&384f89b9&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Pulsar device opened successfully.'}

#### 4. Define and register a callback to handle responses and notifications from Pulsar

To handle responses and notifications from Pulsar, a callback function must be defined and registered. This function will be invoked every time the Pulsar sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(pulsar_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Pulsar.onEvent(callback_function)`` method.

In [5]:
# Define callback function
def callback_function(pulsar_message: dict, system_message: dict) -> None:
    
    if pulsar_message != None:

        # Print message
        print(">> New message from PULSAR:")
        print(pulsar_message)
    
    if system_message != None:

        # Print message
        print(">> New message from the SYSTEM:")
        print(system_message)

In [6]:
# Register callback function
pulsar.onEvent(callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 5. Define a function to generate transaction IDs

All the request messages sent to the Pulsar from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[1, 65535]``.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [7]:
#Auxiliary code to generate IDs.

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

#### 6. Test communication with a basic command such as ``GET USB STRING``

The ``GET USB STRING`` command allows you to retrieve various string values related to the product, such as the manufacturer, product name, serial number, firmware version, and hardware version. This command serves as a basic test to verify the communication and functionality with the Pulsar device. By successfully receiving the expected string values, you can ensure that the communication between the PC host and the Pulsar device is functioning as expected.

In [8]:
# GET USB STRING - Get Manufacturer
r = pulsar.getUsbString(getId(), GetUsbStringSubCommand.MANUFACTURER)

# GET USB STRING  - Get Product Name
r = pulsar.getUsbString(getId(), GetUsbStringSubCommand.PRODUCT_NAME)

# GET USB STRING  - Get Serial Number
r = pulsar.getUsbString(getId(), GetUsbStringSubCommand.SERIAL_NUMBER)

# GET USB STRING  - Firmware Version
r = pulsar.getUsbString(getId(), GetUsbStringSubCommand.FW_VERSION)

# GET USB STRING  - Hardware Version
r = pulsar.getUsbString(getId(), GetUsbStringSubCommand.HW_VERSION)

>> New message from PULSAR:
{'id': 1, 'command': 96, 'name': 'GET USB STRING', 'length': 12, 'message': 'MN-Binho LLC'}
>> New message from PULSAR:
{'id': 2, 'command': 96, 'name': 'GET USB STRING', 'length': 15, 'message': 'PR-Binho Pulsar'}
>> New message from PULSAR:
{'id': 3, 'command': 96, 'name': 'GET USB STRING', 'length': 35, 'message': 'SN-6CAA2A4ED8AB8A598C7E7777A8771F92'}
>> New message from PULSAR:
{'id': 4, 'command': 96, 'name': 'GET USB STRING', 'length': 8, 'message': 'FW-3.1.0'}
>> New message from PULSAR:
{'id': 5, 'command': 96, 'name': 'GET USB STRING', 'length': 4, 'message': 'HW-B'}


## I2C Protocol API

The I2C Protocol API methods are described below.

``Pulsar.setI2cSpiUartBusVoltage(id, bus_voltage_in_mV)``

This method supplies the indicated voltage to the bus in mV, ranging from 1200 mV up to 3300 mV. 

- ``id: int`` : A 2-byte integer that represents the transfer ID.
- ``bus_voltage_in_mV: c_int16 ``: The voltage parameter is a 2-byte integer in the range [1200, 3300].

In [9]:
pulsar.setI2cSpiUartBusVoltage(getId(), 3300)

{'module': 1,
 'opcode': 0,
 'message': 'SET I2C SPI UART BUS VOLTAGE requests success'}

>> New message from PULSAR:
{'id': 6, 'command': 100, 'name': 'SET I2C-SPI-UART BUS VOLTAGE', 'result': 'SYS_NO_ERROR'}


``Pulsar.i2cSetParameters(id, cancelTransfer=0x00, baudrate=0x00)``

This method sets the I2C transfers baud rate and allows canceling the current transfer if necessary.

  - ``id: int`` : A 2-byte integer that represents the transfer ID.
  - ``cancelTransfer: int`` : A subcommand that allows canceling the last I2C transfer if it is still running. This parameter can take two possible values:

    - ``0x00:`` Don't cancel current transfer.
    - ``0x01:`` Cancel current transfer.

- ``baudrate: int`` : This parameter represents the I2C SCL frequency in Hz. Currently, the maximum allowed value is 1000000, which corresponds to 1 MHz.

In [10]:
pulsar.i2cSetParameters(getId(), baudrate=1000000)

{'module': 1, 'opcode': 0, 'message': 'I2C SET PARAMETERS requests success'}

>> New message from PULSAR:
{'id': 7, 'command': 33, 'name': 'I2C SET PARAMETERS', 'completed': 0, 'cancelTransfer': 0, 'baudrate': 32, 'divider': 9}


``Pulsar.i2cSetPullUpResistors(id, pullUpResistorsValue)``

This method sets the I2C pull up resistors value for the SDA and SCL signals.

  - ``id: int`` : A 2-byte integer that represents the transfer ID.
  - ``pullUpResistorsValue: I2cPullUpResistorsValue`` : This parameter indicates the desire pull up resistors value to be set in the SDA and SCL lines. This parameter can take the following values:

    - ``I2cPullUpResistorsValue.I2C_PULLUP_10kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_4_7kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_3_3kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_2_2kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_1_5kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_1kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_680Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_470Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_330Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_220Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_150Ohm``

  Note: By default the pull up resistors are set to 10 kOhms

In [11]:
pulsar.i2cSetPullUpResistors(getId(), I2cPullUpResistorsValue.I2C_PULLUP_4_7kOhm)

{'module': 1,
 'opcode': 0,
 'message': 'I2C SET PULL UP RESISTORS requests success'}

>> New message from PULSAR:
{'id': 8, 'command': 40, 'name': 'I2C SET PULL UP RESISTORS', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'I2C_NO_ERROR', 'driver_error': 'POTENTIOMETER_SET_VALUE_NO_ERROR'}


``Pulsar.i2cWrite(id, slaveAddress, registerAddress: list, data: list)``

This method is used to request to the the Pulsar device to perform an I2C write transfer. The
I2C write transfer starts with a START condition and ends with a STOP condition.

  - ``id: int`` : It is a 2-bytes integer that represent the transfer id.
  - ``slavAddress: c_uint8`` : I2C slave static address.
  - ``registerAddress: list `` : A Python list that contains the memory/register address of the I2C slave's internal memory, to which the data will be written. The list holds bytes, and can have a length from 0 bytes up to 4 bytes. If the list is empty, the Pulsar will ignore it and write only the data payload.
  - ``data: list`` : A Python list that contains the I2C data to be transferred in the I2C write operation. The list holds bytes elements, and the maximum length is 1024 bytes.

In [12]:
SLAVE_ADDRESS   = 0x50
SLAVE_REGISTER = [0,0]   # [reg, page]  
data = [44 for i in range(1,112)]

pulsar.i2cWrite(getId(), SLAVE_ADDRESS, SLAVE_REGISTER, data)

{'module': 1, 'opcode': 0, 'message': 'I2C WRITE requests success'}

>> New message from PULSAR:
{'id': 9, 'command': 34, 'name': 'I2C WRITE', 'status': 'NO_TRANSFER_ERROR', 'payloadLength': 111, 'payload': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

``Pulsar.i2cReadFrom(id, slaveAddress, registerAddress: list, requestDataLength)``

This method is used to request the USB device to perform an I2C read operation, preceded by an I2C write transfer to send the register/memory address from which the data will be read. A R-Start condition is performed between the I2C write and I2C read transfers.

  - ``id: int`` : A 2-byte integer that represent the transaction ID.
  - ``slaveAddress: c_uint8`` : I2C slave static address.
  - ``registerAddress: list `` : A Python list that contains the memory/register address of the I2C slave's internal memory, from which the data will be read. The list holds bytes, and it can have a length from 0 bytes up to 4 bytes. If the list is empty, the Pulsar will ignore it and only write the data payload.
  - ``requestDataLength: int`` : Length of the data to be read. The maximum value is 1024 bytes.

In [13]:
SLAVE_ADDRESS =  0x50
SLAVE_REGISTER = [0,0]   # [reg, page]  
LENGTH = 128

pulsar.i2cReadFrom(getId(), SLAVE_ADDRESS, SLAVE_REGISTER, LENGTH)

{'module': 1, 'opcode': 0, 'message': 'I2C READ FROM requests success'}

>> New message from PULSAR:
{'id': 10, 'command': 39, 'name': 'I2C READ FROM', 'status': 'NO_TRANSFER_ERROR', 'payloadLength': 128, 'data': [44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55]}


``Pulsar.i2cWriteNonStop(id, slaveAddress, registerAddress: list, data: list)``

This method is used to request the the USB device to perform an I2C write operation that starts with a START condition but the STOP condition is not performed after the last byte.

  - ``id: int`` : A 2-byte integer that represents the transaction ID.
  - ``slaveAddress: c_uint8`` : I2C slave static address.
  - ``registerAddress: list `` : A Python list that contains the memory/register address of the I2C slave's internal memory, to which the data will be written.The list holds bytes, and it can have a length from 0 bytes up to 4 bytes. If the list is empty, the Pulsar will ignore it and write only the data payload.
  - ``data: list`` : A Python list that contains the I2C data to be transferred in the I2C write operation. The list holds bytes elements, and the maximum length is 1024 bytes.

In [14]:
SLAVE_ADDRESS   = 0x50
SLAVE_REGISTER = [0,0]   # [reg, page]  
data = [55 for i in range(1,129)]

pulsar.i2cWriteNonStop(getId(), SLAVE_ADDRESS, SLAVE_REGISTER, data)

{'module': 1, 'opcode': 0, 'message': 'I2C WRITE NON STOP requests success'}

>> New message from PULSAR:
{'id': 11, 'command': 38, 'name': 'I2C WRITE WITHOUT STOP', 'status': 'NO_TRANSFER_ERROR', 'payloadLength': 128, 'payload': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

``Pulsar.i2cReadFrom(id, slaveAddress, registerAddress: list, requestDataLength)``

To check the correct execution of the ``i2cWriteNonStop`` command, an ``i2cReadFrom`` command is performed, reading 70 bytes from the slave register [0, 0]

In [15]:
SLAVE_ADDRESS =  0x50
SLAVE_REGISTER = [0,0]   # [reg, page]  
LENGTH = 70

pulsar.i2cReadFrom(getId(), SLAVE_ADDRESS, SLAVE_REGISTER, LENGTH)

{'module': 1, 'opcode': 0, 'message': 'I2C READ FROM requests success'}

>> New message from PULSAR:
{'id': 12, 'command': 39, 'name': 'I2C READ FROM', 'status': 'NO_TRANSFER_ERROR', 'payloadLength': 70, 'data': [55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55]}


``Pulsar.i2cRead(id, slaveAddress, requestDataLength)``

This method is used to request the USB device to perform an I2C read operation that starts with a START condition, but the STOP condition is not performed after the last byte.


  - ``id: int`` : A 2-byte integer that represent the transaction ID.
  - ``slaveAddress: c_uint8`` : I2C slave static address.
  - ``requestDataLength: int`` : Length of data to be read. The maximum value is 1024 bytes.

In [16]:
SLAVE_ADDRESS =  0x50  
LENGTH = 70

pulsar.i2cRead(getId(), SLAVE_ADDRESS,LENGTH)

{'module': 1, 'opcode': 0, 'message': 'I2C READ requests success'}

>> New message from PULSAR:
{'id': 13, 'command': 35, 'name': 'I2C READ', 'status': 'NO_TRANSFER_ERROR', 'payloadLength': 70, 'data': [55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


## Close communication

Use the ``Pulsar.close()`` method to end the communication with the Pulsar device and release the used memory in the background like threads and so on.

In [17]:
# Close the communication with the Pulsar device.
pulsar.close()

{'module': 0, 'opcode': 0, 'message': 'Communication closed successfully.'}